#### **Análise de negócios**

#### **Observações**

#### **Conteúdo - Bases e Notebook da aula**

Github:  

https://github.com/FIAP/Pos_Tech_DTAT/tree/Analise-de-Negocios/Analise%20de%20Negocios  

Série Histórica de Preços de Combustíveis e de GLP:  

https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

#### **Importação de pacotes e bibliotecas**

In [ ]:
# Importar biblioteca completa
import pandas as pd
import os
import requests
import zipfile
from time import sleep

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv

#### **Funções (def)**

In [ ]:
# Função para baixar com validação

def baixar_com_retry(url, caminho, tentativas=3):

    for tentativa in range(1, tentativas + 1):
        try:
            with requests.get(url, stream=True, timeout=120) as r:
                if r.status_code == 404:
                    return None
                
                r.raise_for_status()
                total = int(r.headers.get('content-length', 0))
                baixado = 0

                with open(caminho, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            baixado += len(chunk)

                if total > 0 and baixado < total * 0.9:
                    raise Exception("Download incompleto")
                return caminho
            
        except Exception as e:
            print(f"⚠ Erro na tentativa {tentativa}/{tentativas}: {e}")
            sleep(3)

    print(f"❌ Falha definitiva ao baixar {url}")
    return None

In [ ]:
# Função para baixar arquivo

def baixar_arquivo(url_zip, destino):
    nome_zip = os.path.basename(url_zip)
    caminho_zip = os.path.join(destino, nome_zip)
    print(f"🔽 Tentando baixar ZIP: {nome_zip}")
    zip_baixado = baixar_com_retry(url_zip, caminho_zip)

    if zip_baixado is None:
        url_csv = url_zip.replace(".zip", ".csv")
        nome_csv = os.path.basename(url_csv)
        caminho_csv = os.path.join(destino, nome_csv)
        print(f"🟡 ZIP não encontrado, tentando CSV: {nome_csv}")

        csv_baixado = baixar_com_retry(url_csv, caminho_csv)

        if csv_baixado:
            print(f"✔ Download concluído (CSV): {nome_csv}\n")
        else:
            print(f"❌ Nenhum arquivo encontrado para {nome_zip}\n")
        return

    try:
        with zipfile.ZipFile(caminho_zip, "r") as zip_ref:
            zip_ref.extractall(destino)
        print(f"📦 Arquivo extraído: {nome_zip}\n")
        os.remove(caminho_zip)
        
    except Exception as e:
        print(f"❌ Erro ao extrair {nome_zip}: {e}\n")

In [ ]:
def baixar_arquivos(urls, destino):
    for url in urls:
        nome_arquivo = os.path.basename(url)
        print(f"Baixando {nome_arquivo}...")
        baixar_arquivo(url, destino)

#### **Credenciais**

In [ ]:
load_dotenv()

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER")
senha_pg = os.getenv("POSTGRES_PASSWORD")
host_pg = os.getenv("POSTGRES_HOST")
porta_pg = os.getenv("POSTGRES_PORT")
banco_pg = os.getenv("POSTGRES_DB")

#### **Aula 1 - Processos e formas de análise**

#### **Aula 2 - Ligação com bancos de dados**

In [ ]:
# Exportar os dados da ANP 

anos = range(2004, 2025)
periodos = ["01", "02"]
base_url = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/"
pasta_destino = "combustiveis_automotivos"

os.makedirs(pasta_destino, exist_ok=True)

urls = []
for ano in anos:
    for p in periodos:
        if ano == 2022 and p == "01":
            urls.append(f"{base_url}precos-semestrais-ca.zip")
        else:
            urls.append(f"{base_url}ca-{ano}-{p}.zip")

baixar_arquivos(urls, pasta_destino)